# Step 1: Mounting Google Drive and Importing Libraries


In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/grpo-verified-reasoner
!ls

Mounted at /content/drive
/content/drive/MyDrive/grpo-verified-reasoner
data			      LICENSE	 outputs    unsloth_compiled_cache
grpo_trainer_lora_model       models	 README.md  _unsloth_sentencepiece_temp
huggingface_tokenizers_cache  notebooks  src	    wandb


In [2]:
# Install UV (Faster pip)
!pip install --upgrade -qqq uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 152.8 MB/s eta 0:00:00


In [3]:
!pip -q install -U evalplus

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 13.8 MB/s eta 0:00:00


In [4]:
import os
import subprocess

In [5]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:False"

In [6]:
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"

In [7]:
os.environ["WANDB_PROJECT"] = "mbpp-rl-project"

In [8]:
# Environment Logic (Colab vs Local)
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # Version Matching
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False

    # A100 gets vllm 0.10.2 (Fast), T4 gets 0.9.2 (Stable)
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")

    # Install Everything
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}

# Install TRL
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 19ms
Prepared 1 package in 449ms
Uninstalled 1 package in 204ms
Installed 1 package in 36ms
 - transformers==4.57.3
 + transformers==4.56.2
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1ms
Prepared 1 package in 22ms
Uninstalled 1 package in 1ms
Installed 1 package in 5ms
 - trl==0.24.0
 + trl==0.22.2


In [9]:
import re
import ast
import torch
import wandb
import random
import evalplus
import traceback
import numpy as np
import multiprocessing as mp
from datasets import Dataset
from unsloth import FastLanguageModel
from evalplus.data import get_mbpp_plus
from trl import GRPOConfig, GRPOTrainer
from vllm import SamplingParams

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-04 00:52:22 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
wandb.login()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samyakshrestha (samyakshrestha-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Step 2: Verifying GPU and Environment

In [11]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Torch version: 2.8.0+cu128
CUDA available: True
GPU: NVIDIA H100 80GB HBM3


# Step 3: Loading Base Model and LoRA Adapters

In [12]:
MODEL_PATH = "models/qwen3-4b-sft"

In [13]:
# Load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_PATH,
    max_seq_length = 3072,      # Aligned with GRPO + schema
    load_in_4bit = False,       # Full precision for RL stability
    fast_inference = True,      # Required for vLLM
    gpu_memory_utilization = 0.8,
)

INFO 01-04 00:54:16 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 01-04 00:54:16 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.10: Fast Qwen3 patching. Transformers: 4.56.2. vLLM: 0.10.2.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.32 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Standby mode is enabled. Changing `gpu_memory_utilization` to 0.925.
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 91.8%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 3072. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up 

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 01-04 00:54:38 [__init__.py:1815] Using max model len 3072
INFO 01-04 00:54:39 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-04 00:54:39 [lora.py:92] `lora_extra_vocab_size` is deprecated and will be removed in v0.12.0. Additional vocabulary support for LoRA adapters is being phased out.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

INFO 01-04 00:56:18 [gpu_worker.py:391] Free memory on device (78.64/79.32 GiB) on startup. Desired GPU memory utilization is (0.9180168293092066, 72.82 GiB). Actual usage is 7.81 GiB for weight, 0.76 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.78 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=67914022297` to fit into requested memory, or `--kv-cache-memory=74161210368` to fully utilize gpu memory. Current kv cache memory in use is 68912266649 bytes.
INFO 01-04 00:56:18 [core.py:218] init engine (profile, create kv cache, warmup model) took 74.36 seconds
INFO 01-04 00:56:19 [llm.py:295] Supported_tasks: ('generate',)
INFO 01-04 00:56:19 [__init__.py:36] No IOProcessor plugins requested by the model
Unsloth: Standby mode is enabled. Pre-sleeping vLLM model to reduce OOMs.
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'attention_norm', 'q_norm', 'layer_norm1', 'post_layernorm', 'norm', 'input_layernorm

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen3ForCausalLM were not initialized from the model checkpoint at unsloth/Qwen3-4B-Base and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing substitution for additional_keys=set()
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'attention_norm', 'q_norm', 'layer_norm1', 'post_layernorm', 'norm', 'input_layernorm', 'norm2', 'ffn_norm', 'layer_norm2', 'post_attention_layernorm', 'k_norm', 'cross_attn_post_attention_layernorm', 'norm1', 'cross_attn_input_layernorm', 'pre_feedforward_layernorm']


Unsloth 2025.12.10 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [14]:
trainable = 0
total = 0
trainable_names = []
for name, p in model.named_parameters():
    n = p.numel()
    total += n
    if p.requires_grad:
        trainable += n
        trainable_names.append(name)

print(f"Trainable params: {trainable:,} / {total:,} = {100*trainable/total:.4f}%")
print("Example trainable params:", trainable_names[:20])

Trainable params: 66,060,288 / 4,088,528,384 = 1.6157%
Example trainable params: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.down_proj

In [15]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

# Step 4: Sanity Check

In [16]:
# This is the same prompt that we used during SFT
system_prompt = """You are a code-generation engine.
You must output your response in the following exact format:
<START_WORKING_OUT>
Concise reasoning steps required to solve the problem.
</END_WORKING_OUT>
<SOLUTION>
Valid Python code only.
</SOLUTION>
Do not output anything outside these tags."""

In [17]:
user_prompt = "Write a Python function that returns the factorial of a number."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
)

In [18]:
# Move the dictionary to GPU manually
inputs = {k: v.to("cuda") for k, v in inputs.items()}

In [19]:
FastLanguageModel.for_inference(model) # Temporarily enable inference mode for the test
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.0, # Deterministic check
    )

In [20]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

In [21]:
print("\n--- MODEL OUTPUT ---")
input_len = inputs["input_ids"].shape[1]
print(tokenizer.decode(output[0][input_len:], skip_special_tokens=True))


--- MODEL OUTPUT ---
<START_WORKING_OUT>
Define a function factorial(n) that calculates the product of all positive integers up to n.
Handle non-positive integers by returning 1 (factorial of 0 and negative is 1).
Implement iterative approach for efficiency.
Return the computed factorial.
</END_WORKING_OUT>
<SOLUTION>
def factorial(n):
    if n <= 0:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result
</SOLUTION>


Comment:  No schema check, extractor, or reward function ever sees the full decoded sequence. They only ever see generated_text.

# Step 6: Defining Output Schema

In [22]:
# Regular expressions for tag validation (case-insensitive)
RE_START = re.compile(r"<START_WORKING_OUT>", re.IGNORECASE)
RE_END   = re.compile(r"</END_WORKING_OUT>", re.IGNORECASE)
RE_SOL   = re.compile(r"<SOLUTION>", re.IGNORECASE)
RE_SOL_END = re.compile(r"</SOLUTION>", re.IGNORECASE)

In [23]:
def validate_schema(text: str) -> tuple[bool, str]:
    """
    Checks whether the model output follows the exact required schema.
    Returns (is_valid, reason).
    """
    if not RE_START.search(text):
        return False, "Missing <START_WORKING_OUT>"
    if not RE_END.search(text):
        return False, "Missing </END_WORKING_OUT>"
    if not RE_SOL.search(text):
        return False, "Missing <SOLUTION>"
    if not RE_SOL_END.search(text):
        return False, "Missing </SOLUTION>"

    # Optional: check order consistency
    start_idx = RE_START.search(text).start()
    sol_idx   = RE_SOL.search(text).start()
    if sol_idx < start_idx:
        return False, "Tag order incorrect (<SOLUTION> before reasoning block)."

    return True, "Schema valid"

In [24]:
# Run a sanity test using the previous decoded output
is_valid, reason = validate_schema(decoded)
print("Schema Check:", is_valid, "|", reason)

Schema Check: True | Schema valid


# Step 7: Solution Extraction

In [25]:
# Regex to extract the code block between <SOLUTION> ... </SOLUTION>
RE_SOLUTION = re.compile(r"<SOLUTION>\s*(.*?)\s*</SOLUTION>", re.IGNORECASE | re.DOTALL)

In [26]:
def extract_solution(text: str) -> tuple[str | None, str]:
    """
    Extracts the Python code inside <SOLUTION> tags.
    Returns (code, status) where:
        code   -> the extracted string or None if failed
        status -> textual reason (for debugging)
    """
    match = RE_SOLUTION.search(text)
    if not match:
        return None, "No <SOLUTION> block found."

    code = match.group(1).strip()
    if not code:
        return None, "Empty <SOLUTION> block."

    # Syntax check via Python's AST parser
    try:
        ast.parse(code)
    except SyntaxError as e:
        return None, f"Syntax error in code: {e}"

    return code, "Valid Python code extracted."

In [27]:
# Calculate where the prompt ends
input_len = inputs["input_ids"].shape[1]

In [28]:
# Decode ONLY the new tokens (The Assistant's reply)
generated_text = tokenizer.decode(output[0][input_len:], skip_special_tokens=True)

In [29]:
# Now run the check on ONLY the generated text
code, status = extract_solution(generated_text) # Use the new variable
print("Status:", status)

Status: Valid Python code extracted.


In [30]:

# Show snippet of the extracted code
if code:
    print("\n--- Extracted Python Code ---\n")
    print(code)


--- Extracted Python Code ---

def factorial(n):
    if n <= 0:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result


# Step 8: Verifier Integration (EvalPlus MBPP+)

In [31]:
# Load MBPP+ tasks as a dict: {task_id: problem_dict}
MBPP_TASKS = get_mbpp_plus()

print(f"Loaded MBPP+ tasks: {len(MBPP_TASKS)}")

Loaded MBPP+ tasks: 378


In [32]:
# Quick peek at one task to confirm fields & shape
sample_task_id = next(iter(MBPP_TASKS.keys()))
sample_task = MBPP_TASKS[sample_task_id]

print("\nSample Task ID:", sample_task_id)
print("Keys:", list(sample_task.keys()))
print("\nPrompt (first 400 chars):\n", sample_task["prompt"][:400])


Sample Task ID: Mbpp/2
Keys: ['task_id', 'prompt', 'entry_point', 'canonical_solution', 'base_input', 'atol', 'plus_input', 'contract', 'assertion']

Prompt (first 400 chars):
 """
Write a function to find the shared elements from the given two lists.
assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))
"""



In [33]:
# Different EvalPlus versions may store tests under slightly different keys,
# so we normalize via a helper (used later in reward function).
def get_tests_from_task(task: dict) -> list[str]:
    """
    Extracts MBPP test assertions from a task.
    Supports both list-based and string-based formats.
    """
    # Case 1: already a list of assertions
    for k in ("test_list", "tests", "plus_tests", "base_tests"):
        if k in task and task[k]:
            return list(task[k])

    # Case 2: single multiline assertion string (MBPP+ common case)
    if "assertion" in task and task["assertion"]:
        lines = task["assertion"].strip().splitlines()
        return [line for line in lines if line.strip()]

    raise KeyError(f"No tests found in task keys: {list(task.keys())}")

# Step 9: Defining Helper Functions

In [34]:
def _exec_code_and_tests_worker(code: str, tests: list[str], queue: mp.Queue) -> None:
    """
    Runs model code + tests.
    CRITICAL FEATURES:
    1. Runs ALL tests (Partial Credit).
    2. Catches ALL exceptions (Robustness).
    3. Truncates error logs (IPC Safety).
    """
    try:
        # Create the "Main Desk" (Environment)
        env = {"__builtins__": __builtins__}

        # Run the User's Code into 'env'
        exec(code, env, env)

        passed_count = 0
        total_tests = len(tests)
        first_error = None

        # Run ALL Test Cases
        for t in tests:
            try:
                exec(t, env, env)
                passed_count += 1
            except Exception:
                # Capture only the FIRST error to save bandwidth
                if first_error is None:
                    # Get the full traceback
                    tb = traceback.format_exc()
                    # SOPHIA'S FIX: Truncate to 500 chars to prevent IPC deadlock
                    first_error = tb[:500] + "\n...[TRUNCATED]..." if len(tb) > 500 else tb
                # Continue to the next test!
                continue

        # Mission Complete: Return the score
        queue.put((passed_count, total_tests, first_error))

    except Exception:
        # Catch syntax errors or crashes in the main code body
        tb = traceback.format_exc()
        truncated_error = tb[:500] + "\n...[TRUNCATED]..." if len(tb) > 500 else tb
        queue.put((0, len(tests), truncated_error))

In [35]:
def run_mbpp_tests(code: str, task: dict, timeout_s: float = 2.0) -> tuple[int, int, str | None]:
    """
    Executes tests and returns (passed_count, total_count, first_error).
    """
    tests = get_tests_from_task(task)
    if not tests:
        return 0, 0, "No tests found."

    ctx = mp.get_context("fork")
    q = ctx.Queue()
    p = ctx.Process(target=_exec_code_and_tests_worker, args=(code, tests, q))
    p.start()
    p.join(timeout_s)

    if p.is_alive():
        p.terminate()
        p.join()
        return 0, len(tests), f"Timeout after {timeout_s:.1f}s"

    if q.empty():
        return 0, len(tests), "No result returned from worker."

    passed_count, total_count, err = q.get()
    return passed_count, total_count, err

# Step 10: Defining Reward Functions

In [36]:
def format_reward_func(completions, **kwargs) -> list[float]:
    """
    Rewards the model for strictly following the XML schema.
    Args:
        completions: List of generated strings from the model.
    Returns:
        List of rewards (0.1 for valid schema, 0.0 for invalid).
    """
    rewards = []
    for completion in completions:
        # Uses your existing validator from Step 6
        is_valid, _ = validate_schema(completion)
        rewards.append(0.1 if is_valid else 0.0)
    return rewards

In [37]:
def reasoning_reward_func(completions, **kwargs) -> list[float]:
    """
    Rewards the model for reasoning, but PENALIZES rambling.

    Structure:
    - Base Reward: Up to +0.15 for writing ~500 chars of thought.
    - Penalty 1: -0.05 if length > 700 chars.
    - Penalty 2: Additional -0.05 (Total -0.10) if length > 1000 chars.
    """
    rewards = []
    for completion in completions:
        # Regex to find the reasoning block specifically
        match = re.search(r"<START_WORKING_OUT>(.*?)</END_WORKING_OUT>", completion, re.DOTALL | re.IGNORECASE)

        if match:
            reasoning_content = match.group(1).strip()
            length = len(reasoning_content)

            # 1. Base Positive Reward (Capped at 0.15 instead of 0.2)
            # We want it to think, but not think too much.
            score = min(0.15, (length / 500.0) * 0.15)

            # 2. Apply Penalties (The "Anti-Ramble" Mechanism)
            if length > 700:
                score -= 0.05  # First warning shot
            if length > 1000:
                score -= 0.05  # Serious penalty (Total -0.10)

            rewards.append(score)
        else:
            rewards.append(0.0)

    return rewards

In [38]:
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    Rewards the model based on the PERCENTAGE of tests passed.
    Includes a "Clean Sweep Bonus" for 100% completion.
    """
    rewards = []
    for prompt, completion, task_data in zip(prompts, completions, answer):
        code, status = extract_solution(completion)
        if not code:
            rewards.append(0.0)
            continue

        passed, total, err = run_mbpp_tests(code, task_data)

        if total == 0:
            rewards.append(0.0)
            continue

        # CALCULATE SCORE: Fraction of tests passed
        score = passed / total

        # VICTORY BONUS: If 100% passed, add +0.1 bonus
        # This differentiates "perfect" from "lucky" and prevents settling
        if passed == total:
            score += 0.1

        rewards.append(score)

    return rewards

# Step 11: Dataset Formatting and Unit Testing

In [39]:
# Clean the Data
# The raw MBPP+ dataset has inconsistent schemas (some fields are lists, some are None).
# We fix this by extracting ONLY what we need: the test cases.
dict_data = []

In [40]:
for task_id, task_data in MBPP_TASKS.items():
    # Extract the test cases using our helper from Step 8
    # This handles the "messy" parsing right now, so the Dataset is clean.
    try:
        tests = get_tests_from_task(task_data)
    except KeyError:
        # If a task is broken/empty, skip it to prevent crashes
        print(f"Skipping task {task_id}: No tests found.")
        continue

    # Create a CLEAN 'answer' dictionary
    # This guarantees every row has the exact same structure.
    # This prevents the "ArrowInvalid" error.
    clean_answer = {
        "task_id": str(task_id),
        "test_list": tests  # Always a List of Strings
    }

    # Append to our list
    dict_data.append({
        "prompt": task_data["prompt"],
        "answer": clean_answer
    })

In [41]:
# Creating a Hugging Face compatible dataset
dataset = Dataset.from_list(dict_data)

In [42]:
print("Dataset Features:", dataset.features)
print("Sample Row Answer Keys:", dataset[0]["answer"].keys())

Dataset Features: {'prompt': Value('string'), 'answer': {'task_id': Value('string'), 'test_list': List(Value('string'))}}
Sample Row Answer Keys: dict_keys(['task_id', 'test_list'])


In [43]:
# Pick the 2nd task again for consistency
task = dataset[2]["answer"] # We grab it from our NEW dataset column
prompt = dataset[2]["prompt"]

In [44]:
prompt

'"""\nWrite a function to find the n largest integers from a given list of numbers, returned in descending order.\nassert heap_queue_largest( [25, 35, 22, 85, 14, 65, 75, 22, 58],3)==[85, 75, 65]\n"""\n'

In [45]:
# Build the prompt structure
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]

In [46]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
)
inputs = {k: v.to("cuda") for k, v in inputs.items()}

In [47]:
# Generate
FastLanguageModel.for_inference(model)
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.0,
    )

In [48]:
# Slice to get only the generated text
input_len = inputs["input_ids"].shape[1]
generated_text = tokenizer.decode(output[0][input_len:], skip_special_tokens=True)

In [49]:
print(generated_text)

<START_WORKING_OUT>
Problem: Find n largest integers from a list, return in descending order.
Approach: Use heapq.nlargest which returns n largest elements in order from largest to smallest.
Parameters: List of numbers, integer n.
Return: List of n largest numbers in descending order.
</END_WORKING_OUT>
<SOLUTION>
import heapq

def heap_queue_largest(nums, n):
    """
    Return the n largest numbers from nums in descending order.
    
    Args:
        nums: List of numbers (integers or floats)
        n: Number of largest elements to return
        
    Returns:
        List of n largest numbers in descending order
    """
    if n <= 0:
        return []
    if n >= len(nums):
        nums_sorted = sorted(nums, reverse=True)
        return nums_sorted[:n]
    return heapq.nlargest(n, nums)
</SOLUTION>


In [50]:
# CRITICAL PART: Testing the Reward Functions
# The Reward Functions expect LISTS (Batches), so we wrap our single item in a list.
# This simulates a batch size of 1.
batch_prompts = [prompt]
batch_completions = [generated_text]
batch_answers = [task] # This is the "answer" column data

In [51]:
# 1. Test Format Reward
r_format = format_reward_func(completions=batch_completions)
print(f"Format Reward (Expect 0.1): {r_format[0]}")

Format Reward (Expect 0.1): 0.1


In [52]:
# 2. Test Reasoning Reward
r_reason = reasoning_reward_func(completions=batch_completions)
print(f"Reasoning Reward (Expect 0.0-0.15): {r_reason[0]:.4f}")

Reasoning Reward (Expect 0.0-0.15): 0.0795


In [53]:

# 3. Test Correctness Reward (The complex one)
# Note: We pass 'answer' explicitly, just like the Trainer will.
r_correct = correctness_reward_func(
    prompts=batch_prompts,
    completions=batch_completions,
    answer=batch_answers
)
print(f"Correctness Reward (Expect 1.0 or 0.0): {r_correct[0]}")

Correctness Reward (Expect 1.0 or 0.0): 1.1


In [54]:
if r_format[0] > 0 and (r_correct[0] == 0.0 or r_correct[0] == 1.1):
    print(" SUCCESS: All reward functions accepted the inputs and returned scores.")
    print(" The plumbing is connected correctly.")
else:
    print(" FAIL: Something returned an unexpected format.")

 SUCCESS: All reward functions accepted the inputs and returned scores.
 The plumbing is connected correctly.


# Step 12: Apply Chat Template

In [55]:
def apply_chat_template(row):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row["prompt"]}
    ]

    # "tokenize=False" gives us the raw text string (e.g. "<|system|>...<|user|>...")
    # This is exactly what the GRPOTrainer expects in the 'prompt' column.
    row["prompt"] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return row

In [56]:
# Apply it to the whole dataset
original_prompt = dataset[0]["prompt"]
dataset = dataset.map(apply_chat_template)

print("\n--- BEFORE ---")
print(original_prompt)
print("\n--- AFTER (What the Model Sees) ---")
print(dataset[0]["prompt"])

Map:   0%|          | 0/378 [00:00<?, ? examples/s]


--- BEFORE ---
"""
Write a function to find the shared elements from the given two lists.
assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))
"""


--- AFTER (What the Model Sees) ---
<|im_start|>system
You are a code-generation engine.
You must output your response in the following exact format:
<START_WORKING_OUT>
Concise reasoning steps required to solve the problem.
</END_WORKING_OUT>
<SOLUTION>
Valid Python code only.
</SOLUTION>
Do not output anything outside these tags.<|im_end|>
<|im_start|>user
"""
Write a function to find the shared elements from the given two lists.
assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))
"""
<|im_end|>
<|im_start|>assistant



# Step 13: Setting up GRPO Configurations

In [57]:
# We give the model ample room so it never gets cut off
max_prompt_length = 512
max_completion_length = 2048  # doubled from T4 config

In [58]:
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 0.95,
    top_k = -1,
    seed = 3407,
    temperature = 0.8, # High enough to get diverse answers for GRPO
    stop = ["</SOLUTION>"],
    include_stop_str_in_output = True,
)

In [59]:
# 3. The Trainer Config
training_args = GRPOConfig(
    # Integration
    vllm_sampling_params = vllm_sampling_params, # We use vLLM for speed
    output_dir = "outputs",
    report_to = "wandb",
    run_name = "mbpp-grpo-a100-run3-full",

    # Optimization
    learning_rate = 1e-5,        # Aggressive for short horizon
    beta = 0.005,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",

    # A100 POWER SETTINGS
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    num_generations = 16,             # G=8: Much better stability than G=4

    # Lengths
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,

    # Duration
    num_train_epochs = 2,            # 1 Epoch is safest for RL on small data
    #max_steps = 5,

    # Logging
    logging_steps = 5,
    save_steps = 20,                 # Save more frequently
    use_vllm = True,                 # Explicitly enable vLLM
)

Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 16


# Step 14: Initialize and Run GRPO Trainer

In [60]:
# Select the Reward Functions we defined in Step 10
# These are the "Judges" that will score the model's outputs.
reward_functions = [
    format_reward_func,       # Did it use <START_WORKING_OUT> and <SOLUTION>? (0.1)
    reasoning_reward_func,    # Did it write ~500 chars of thought? (0.2)
    correctness_reward_func   # Did the code actually pass the tests? (1.0)
]

In [61]:
# Initialize the Trainer
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = reward_functions,
    args = training_args,         # The A100 Config we just built
    train_dataset = dataset,      # The dataset with the Chat Template applied
)

In [62]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 378 | Num Epochs = 2 | Total steps = 188
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / format_reward_func / mean,rewards / format_reward_func / std,rewards / reasoning_reward_func / mean,rewards / reasoning_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
5,0.003200,0.995726,0.188900,693.800000,693.800000,693.800000,1.000000,0.000000,0.000000,0.000000,0.649213,0.099688,0.002500,0.076663,0.032918,0.819375,0.417288
10,0.002700,1.093298,0.242669,776.400000,776.400000,776.400000,1.000000,0.000000,0.000000,0.000000,0.536763,0.099375,0.003507,0.080381,0.031195,0.913542,0.372759
15,0.003300,1.008403,0.310461,359.400000,359.400000,359.400000,1.000000,0.000000,0.000000,0.000000,0.661359,0.099688,0.002500,0.088403,0.034427,0.820313,0.434723
20,0.002300,0.989610,0.263610,1478.200000,1478.200000,1478.200000,1.000000,0.000000,0.000000,0.000000,0.454509,0.098750,0.008507,0.093152,0.033721,0.797708,0.457775
25,0.002300,1.127841,0.242982,455.400000,455.400000,455.400000,1.000000,0.000000,0.000000,0.000000,0.458946,0.099375,0.005000,0.098362,0.034652,0.930104,0.344904
30,0.004600,0.854438,0.266999,1175.987500,822.400000,1181.600000,0.996875,50.400000,50.400000,50.400000,0.923881,0.098438,0.007768,0.101833,0.033994,0.654167,0.490555
35,0.002100,1.015911,0.275771,837.200000,837.200000,837.200000,1.000000,0.000000,0.000000,0.000000,0.414263,0.099688,0.002500,0.105858,0.033936,0.810365,0.442271
40,0.004700,0.887793,0.307196,1790.600000,1790.600000,1790.600000,1.000000,0.000000,0.000000,0.000000,0.930803,0.096875,0.015148,0.106543,0.036287,0.684375,0.466817
45,0.002000,1.001083,0.246510,1411.000000,1411.000000,1411.000000,1.000000,0.000000,0.000000,0.000000,0.397159,0.098437,0.007768,0.107958,0.035586,0.794688,0.471774
50,0.002000,1.093301,0.345522,1736.600000,1736.600000,1736.600000,1.000000,0.000000,0.000000,0.000000,0.392302,0.097500,0.015268,0.114625,0.036817,0.881176,0.386312


1.5707963267948966
((3.0, -2.2831853071795867), (-1.960930862590836-2.2704074859237844j))
26
8
2


KeyboardInterrupt: 

# Step 15: Sanity Check

Let us now check the model that we just trained!

In [ ]:
# Switch to Inference Mode
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
test_question = "Write a function to find the volume of a sphere given its radius."

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": test_question},
]

In [ ]:
# Tokenize (Exactly as before)
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
)
inputs = {k: v.to("cuda") for k, v in inputs.items()}

In [ ]:
# Generate
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.8, # Slight creativity to encourage reasoning
    )

In [ ]:
# 5. Decode (Slicing input_len just like before)
input_len = inputs["input_ids"].shape[1]
generated_text = tokenizer.decode(output[0][input_len:], skip_special_tokens=True)

print("\n=== FINAL MODEL OUTPUT ===")
print(generated_text)


=== FINAL MODEL OUTPUT ===
<START_WORKING_OUT>
The formula for the volume of a sphere is V = (4/3) * π * r³.
We need a function that takes the radius r as input and returns the volume.
The function should handle floating-point numbers for the radius.
We'll use Python's math module for the value of π (math.pi).
The calculation involves cubing the radius (r**3) and then multiplying by (4/3) * π.
We'll return the result as a float.
</END_WORKING_OUT>
<SOLUTION>
import math

def sphere_volume(radius):
    """
    Calculate the volume of a sphere given its radius.
    
    Parameters:
    radius (float): The radius of the sphere.
    
    Returns:
    float: The volume of the sphere.
    """
    if radius < 0:
        raise ValueError("Radius cannot be negative")
    return (4/3) * math.pi * (radius ** 3)
</SOLUTION>


# Step 16: Saving the Model

In [ ]:
MODEL_OUT = "models/qwen3-4b-grpo-final"

In [ ]:
model.save_lora(MODEL_OUT)

In [ ]:
tokenizer.save_pretrained(MODEL_OUT)

('models/qwen3-4b-grpo-final/tokenizer_config.json',
 'models/qwen3-4b-grpo-final/special_tokens_map.json',
 'models/qwen3-4b-grpo-final/chat_template.jinja',
 'models/qwen3-4b-grpo-final/vocab.json',
 'models/qwen3-4b-grpo-final/merges.txt',
 'models/qwen3-4b-grpo-final/added_tokens.json',
 'models/qwen3-4b-grpo-final/tokenizer.json')